# **Baseline Simple MLP with just MSE**

## **Running the models using the 'modelling' package**

A notebook through which different modelling configurations can be ran, using the ``modelling`` package. It follows the steps of:
- preparing packages;
- setting "global" variables;
- getting the data;
- defining hyperparameters;
- running a grid search and/or training a model; and
- evaluation.
In the modelling package, variations can be made to the models and training functions to experiment. Don't forget to restart the notebook after making changes there.



For loading models, go to the ``src/results/models``:
- Baseline NO2 2017 with MLP and MSE loss: ``best_mlp_no2_baseline.pth``
- Exp 1: NO2 2017 with MLP and option 1 simple physics loss: ``best_mlp_no2_adjusted_dist.pth`` (naming because I updated the distance between T and B)


In [1]:
print("Starting script...")


from modelling.MLP import BasicMLP
from modelling import *


import optuna
import threading
import os
from pathlib import Path
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset

Starting script...

Running __init__.py for data pipeline...


Modelling package initialized



Use GPU when available

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

Device:  cpu


### **Set "global" variables**

In [3]:
Path.cwd()

PosixPath('/home/rachel/forecasting_smog_PEML/src')

In [4]:
HABROK = bool(0)                  # set to True if using HABROK; it will print
                                  # all stdout to a .txt file to log progress

BASE_DIR = Path.cwd().parents[0] # set it to the root directory of the project, not src
MODEL_PATH = BASE_DIR /"src" / "results" / "models"
MINMAX_PATH = BASE_DIR  / "data" / "data_combined" / "pollutants_minmax.csv"

print("BASE_DIR: ", BASE_DIR)
print("MODEL_PATH: ", MODEL_PATH)
print("MINMAX_PATH: ", MINMAX_PATH)

torch.manual_seed(34)             # set seed for reproducibility

N_HOURS_U = 24 * 3               # number of hours to use for input (number of days * 24 hours)
N_HOURS_Y = 24                    # number of hours to predict (1 day * 24 hours)
N_HOURS_STEP = 24                 # "sampling rate" in hours of the data; e.g. 24 
                                  # means sample an I/O-pair every 24 hours
                                  # the contaminants and meteorological vars

# Change this according to the data you want to use
YEARS = [2017]
TRAIN_YEARS = [2017]
VAL_YEARS = [2017]
TEST_YEARS = [2017]

torch.random.manual_seed(34)

BASE_DIR:  /home/rachel/forecasting_smog_PEML
MODEL_PATH:  /home/rachel/forecasting_smog_PEML/src/results/models
MINMAX_PATH:  /home/rachel/forecasting_smog_PEML/data/data_combined/pollutants_minmax.csv


### **Load in data and create PyTorch *Datasets***

In [5]:
# Load in data and create PyTorch Datasets. To tune
# which exact .csv files get extracted, change the
# lists in the get_dataframes() definition

train_input_frames = get_dataframes('train', 'u', YEARS)
train_output_frames = get_dataframes('train', 'y', YEARS)

val_input_frames = get_dataframes('val', 'u', YEARS)
val_output_frames = get_dataframes('val', 'y', YEARS)

test_input_frames = get_dataframes('test', 'u', YEARS)
test_output_frames = get_dataframes('test', 'y', YEARS) 

print("Successfully loaded data")

Imported train_2017_combined_u.csv
Imported train_2017_combined_y.csv
Imported val_2017_combined_u.csv
Imported val_2017_combined_y.csv
Imported test_2017_combined_u.csv
Imported test_2017_combined_y.csv
Successfully loaded data


In [6]:
train_dataset = TimeSeriesDataset(
    train_input_frames,  # list of input training dataframes
    train_output_frames, # list of output training dataframes
    len(TRAIN_YEARS),                   # number of dataframes put in for both
                         # (basically len(train_input_frames) and
                         # len(train_output_frames) must be equal)
    N_HOURS_U,           # number of hours of input data
    N_HOURS_Y,           # number of hours of output data
    N_HOURS_STEP,        # number of hours between each input/output pair
)
val_dataset = TimeSeriesDataset(
    val_input_frames,    # etc.
    val_output_frames,
    len(VAL_YEARS),
    N_HOURS_U,
    N_HOURS_Y,
    N_HOURS_STEP,
)
test_dataset = TimeSeriesDataset(
    test_input_frames,
    test_output_frames,
    len(TEST_YEARS),
    N_HOURS_U,
    N_HOURS_Y,
    N_HOURS_STEP,
)

del train_input_frames, train_output_frames
del val_input_frames, val_output_frames
del test_input_frames, test_output_frames

In [7]:
train_dataset.u

[                           DD   FF        FH        FX       NO2         P  \
 DateTime                                                                     
 2017-08-01 00:00:00  0.166667  0.1  0.111111  0.000000  0.242115  0.562982   
 2017-08-01 01:00:00  0.000000  0.0  0.111111  0.052632  0.223158  0.570694   
 2017-08-01 02:00:00  0.000000  0.0  0.000000  0.000000  0.165911  0.560411   
 2017-08-01 03:00:00  0.277778  0.1  0.000000  0.000000  0.142363  0.555270   
 2017-08-01 04:00:00  0.805556  0.2  0.111111  0.105263  0.156297  0.555270   
 ...                       ...  ...       ...       ...       ...       ...   
 2017-11-16 19:00:00  0.750000  0.2  0.333333  0.210526  0.523871  0.789203   
 2017-11-16 20:00:00  0.972222  0.3  0.333333  0.421053  0.512314  0.814910   
 2017-11-16 21:00:00  0.888889  0.1  0.222222  0.263158  0.232880  0.827763   
 2017-11-16 22:00:00  0.944444  0.2  0.111111  0.105263  0.108123  0.832905   
 2017-11-16 23:00:00  0.861111  0.1  0.222222  0.105

In [8]:
train_dataset.y

[                          NO2
 DateTime                     
 2017-08-01 00:00:00  0.223698
 2017-08-01 01:00:00  0.145496
 2017-08-01 02:00:00  0.275978
 2017-08-01 03:00:00  0.423742
 2017-08-01 04:00:00  0.478721
 ...                       ...
 2017-11-16 19:00:00  0.606502
 2017-11-16 20:00:00  0.456470
 2017-11-16 21:00:00  0.483258
 2017-11-16 22:00:00  0.468784
 2017-11-16 23:00:00  0.473428
 
 [2592 rows x 1 columns]]

In [9]:
len(train_dataset.pairs[0][0])

72

In [10]:
train_dataset.pairs[0][0]

tensor([[0.1667, 0.1000, 0.1111, 0.0000, 0.2421, 0.5630, 0.0000, 0.5367, 0.7269],
        [0.0000, 0.0000, 0.1111, 0.0526, 0.2232, 0.5707, 0.0000, 0.5467, 0.7407],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1659, 0.5604, 0.0000, 0.5067, 0.6898],
        [0.2778, 0.1000, 0.0000, 0.0000, 0.1424, 0.5553, 0.0000, 0.4633, 0.6343],
        [0.8056, 0.2000, 0.1111, 0.1053, 0.1563, 0.5553, 0.0000, 0.4933, 0.6620],
        [0.0000, 0.0000, 0.1111, 0.1053, 0.3135, 0.5681, 0.3000, 0.6200, 0.7593],
        [0.7222, 0.1000, 0.1111, 0.0526, 0.5326, 0.5913, 0.0000, 0.6433, 0.7269],
        [0.7500, 0.1000, 0.1111, 0.1053, 0.5367, 0.5938, 0.0000, 0.6500, 0.7037],
        [0.7222, 0.2000, 0.2222, 0.1053, 0.5172, 0.5964, 0.0000, 0.6733, 0.6574],
        [0.7500, 0.2000, 0.2222, 0.2105, 0.4459, 0.5990, 0.3000, 0.7133, 0.6157],
        [0.6111, 0.2000, 0.2222, 0.1579, 0.3129, 0.6041, 0.0000, 0.7167, 0.6019],
        [0.6111, 0.2000, 0.2222, 0.1579, 0.3478, 0.6067, 0.0000, 0.7133, 0.5926],
        [0.6528,

In [11]:
train_dataset.pairs[0][1]

tensor([[0.1965],
        [0.1501],
        [0.1518],
        [0.2622],
        [0.5524],
        [0.4840],
        [0.3544],
        [0.2754],
        [0.1948],
        [0.1734],
        [0.1505],
        [0.1352],
        [0.0778],
        [0.1184],
        [0.1293],
        [0.1238],
        [0.1043],
        [0.0997],
        [0.0812],
        [0.0823],
        [0.1155],
        [0.0837],
        [0.0570],
        [0.1006]])

In [12]:
# Assuming train_dataset.u[0] is a pandas Index object with column names
column_names = list(train_dataset.u[0])  # Convert Index to list

# Now, find the indices of the columns 'NO2', 'DD', 'FH'
no2_idx = column_names.index('NO2')
dd_idx = column_names.index('DD')
fh_idx = column_names.index('FH')

print("NO2 index: ", no2_idx)
print("DD index (wind direction): ", dd_idx)
print("FH index (Hourly wind speed): ", fh_idx)


NO2 index:  4
DD index (wind direction):  0
FH index (Hourly wind speed):  2


In [13]:
train_dataset.u[0].iloc[:,no2_idx]

DateTime
2017-08-01 00:00:00    0.242115
2017-08-01 01:00:00    0.223158
2017-08-01 02:00:00    0.165911
2017-08-01 03:00:00    0.142363
2017-08-01 04:00:00    0.156297
                         ...   
2017-11-16 19:00:00    0.523871
2017-11-16 20:00:00    0.512314
2017-11-16 21:00:00    0.232880
2017-11-16 22:00:00    0.108123
2017-11-16 23:00:00    0.205120
Name: NO2, Length: 2592, dtype: float64

In [14]:
train_dataset.u[0].iloc[:,dd_idx]

DateTime
2017-08-01 00:00:00    0.166667
2017-08-01 01:00:00    0.000000
2017-08-01 02:00:00    0.000000
2017-08-01 03:00:00    0.277778
2017-08-01 04:00:00    0.805556
                         ...   
2017-11-16 19:00:00    0.750000
2017-11-16 20:00:00    0.972222
2017-11-16 21:00:00    0.888889
2017-11-16 22:00:00    0.944444
2017-11-16 23:00:00    0.861111
Name: DD, Length: 2592, dtype: float64

In [15]:
train_dataset.u[0].iloc[:,fh_idx]

DateTime
2017-08-01 00:00:00    0.111111
2017-08-01 01:00:00    0.111111
2017-08-01 02:00:00    0.000000
2017-08-01 03:00:00    0.000000
2017-08-01 04:00:00    0.111111
                         ...   
2017-11-16 19:00:00    0.333333
2017-11-16 20:00:00    0.333333
2017-11-16 21:00:00    0.222222
2017-11-16 22:00:00    0.111111
2017-11-16 23:00:00    0.222222
Name: FH, Length: 2592, dtype: float64

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
from torch.utils.data import DataLoader

class PhysicsMLP(nn.Module):
    def __init__(self, N_INPUT_UNITS=72, N_HIDDEN_LAYERS=2, N_HIDDEN_UNITS=100, N_OUTPUT_UNITS=24, loss_function="MSE"):
        super(PhysicsMLP, self).__init__()
        self.loss_function = loss_function

        layers = [nn.Linear(N_INPUT_UNITS, N_HIDDEN_UNITS), nn.ReLU()]
        for _ in range(N_HIDDEN_LAYERS):
            layers.append(nn.Linear(N_HIDDEN_UNITS, N_HIDDEN_UNITS))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(N_HIDDEN_UNITS, N_OUTPUT_UNITS))
        self.layers = nn.Sequential(*layers)

        # Compute constant dx, dy using lat/lon coordinates of Tuindorp & Breukelen
        self.dx, self.dy = self.compute_dx_dy()

    def compute_dx_dy(self):
        """Compute fixed spatial distances (dx, dy) between Tuindorp and Breukelen."""
        lat_tuindorp, lon_tuindorp = 52.10503, 5.12448  # Tuindorp coords
        lat_breukelen, lon_breukelen = 52.20153, 4.98741  # Breukelen coords
        R = 6371  # Earth radius in km
        
        x = (lon_breukelen - lon_tuindorp) * (math.pi / 180) * R * math.cos(math.radians((lat_tuindorp + lat_breukelen) / 2))
        y = (lat_breukelen - lat_tuindorp) * (math.pi / 180) * R
        return x, y

    def compute_wind_components(self, u):
        """Compute wind speed components (v_x, v_y) from wind speed and direction."""
        dd_idx = 0  # Wind direction index
        fh_idx = 2  # Wind speed index


        wind_speed = u[:, :, fh_idx] * 3.6  # Convert m/s to km/h
        wind_direction = u[:, :, dd_idx] * 360  # Convert normalized [0,1] to degrees

        v_x = wind_speed * torch.cos(torch.deg2rad(wind_direction))  # X-component
        v_y = wind_speed * torch.sin(torch.deg2rad(wind_direction))  # Y-component
        return v_x, v_y

    def forward(self, u):
        batch_size, seq_length, n_features = u.size()
        outputs = torch.empty((batch_size, seq_length, self.layers[-1].out_features), device=u.device)

        for t in range(seq_length):
            u_t = u[:, t, :]
            y_t = self.layers(u_t)
            outputs[:, t, :] = y_t

        return outputs[:, -24:, :]

    def physics_loss(self, u, y_pred, v_x, v_y, dt):
        """
        Compute physics loss enforcing the advection equation:
        ∂c/∂t + v_x * ∂c/∂x + v_y * ∂c/∂y = 0
        """
        batch_size, seq_len, _ = u.size()
        loss_phy = 0.0
        no2_idx = 4  # NO2 index

        for t in range(1, 24):  
            c_t = y_pred[:, t, :]
            c_t_prev = y_pred[:, t - 1, :]

            # Temporal derivative ∂c/∂t
            dc_dt = (c_t - c_t_prev) / dt

            # Spatial derivatives ∂c/∂x and ∂c/∂y using autograd
            dc_dx = torch.autograd.grad(c_t, u, torch.ones_like(c_t), create_graph=True)[0] / self.dx
            dc_dy = torch.autograd.grad(c_t, u, torch.ones_like(c_t), create_graph=True)[0] / self.dy
            # 16, 72, 9
            # 16 , 72

            # Ensure that dc_dx and dc_dy have shape (batch_size, 72)
            # Squeeze or select the first feature if necessary (from the extra dimension)
            dc_dx = dc_dx[:, :, no2_idx]  # Assuming we want the first feature for simplicity
            dc_dy = dc_dy[:, :, no2_idx]  # Assuming we want the first feature for simplicity

            # print("vx shape",v_x.shape)
            # print("dc_dx shape",dc_dx.shape)
            # print("vy shape",v_y.shape)
            # print("dc_dy shape",dc_dy.shape)
            # print("dc_dt shape",dc_dt.shape)

            # Compute residual of PDE
            residual = dc_dt + v_x * dc_dx + v_y * dc_dy
            loss_phy += torch.mean(residual ** 2)

        return loss_phy / (24 - 2)  # Average over time steps

    def train_model(self, train_loader, val_loader, epochs=50, lr=1e-3, weight_decay=1e-6, lambda_phy=1e-5, device="cpu"):
        self.to(device)
        optimizer = optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay)

        best_val_loss = float("inf")
        best_model_state = None
        dt = 1  # Time step in hours

        for epoch in range(epochs):
            self.train()
            train_loss = 0.0

            for u, y in train_loader:
                u, y = u.to(device), y.to(device)

                u = u.float()  # Ensure float dtype
                u.requires_grad = True  # Ensure gradient tracking is enabled for u
            
                optimizer.zero_grad()
                y_pred = self.forward(u)  # Forward pass should result in tensors that require gradients

                # Compute wind components
                v_x, v_y = self.compute_wind_components(u)

                # Data-driven loss
                loss_data = torch.nn.functional.mse_loss(y_pred, y)

                # Physics loss
                loss_phy = self.physics_loss(u, y_pred, v_x, v_y, dt)

                # print("Loss phy: ", loss_phy)
                # print("Loss data: ", loss_data)

                # Combine losses
                loss = loss_data + lambda_phy * loss_phy
                # print("Total loss: ", loss)

                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            train_loss /= len(train_loader)


            # Validation step
            self.eval()
            val_loss = 0.0

            for u, y in val_loader:
                u, y = u.to(device), y.to(device)

                u = u.float()  # Ensure float dtype
                u.requires_grad = True  # No need for gradients during validation

                y_pred = self.forward(u)
                v_x, v_y = self.compute_wind_components(u)

                loss = torch.nn.functional.mse_loss(y_pred, y)
                val_loss += loss.item()

            val_loss /= len(val_loader)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = self.state_dict()

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f} - Val Loss: {val_loss:.6f}")

        if best_model_state:
            self.load_state_dict(best_model_state)

        return best_val_loss


    def test_model(self, test_loader, min_value=None, max_value=None, device="cpu"):
        self.to(device)
        self.eval()
        mse_loss_fn = nn.MSELoss()
        rmse_loss = 0.0
        smape_loss = 0.0
        total_elements = 0
        epsilon = 1e-6  # To prevent division by zero

        with torch.no_grad():
            for u, y in test_loader:
                u, y = u.to(device), y.to(device)
                output = self.forward(u)

                # Compute SMAPE BEFORE denormalization
                abs_diff = torch.abs(y - output)
                sum_abs = torch.abs(y) + torch.abs(output) + epsilon  # Avoid division by zero
                smape_batch = torch.sum(2 * abs_diff / sum_abs).item()

                smape_loss += smape_batch
                total_elements += y.numel()  # Count total number of elements

                # Denormalize for RMSE and MSE calculation
                if min_value is not None and max_value is not None:
                    output = output * (max_value - min_value) + min_value
                    y = y * (max_value - min_value) + min_value

                mse_loss = mse_loss_fn(output, y)
                rmse_loss += torch.sqrt(mse_loss).item()

        # Final loss calculations
        rmse_loss /= len(test_loader)  # Average over batches
        smape_loss = (smape_loss / total_elements) * 100  # Average over all elements and convert to %

        print(f"Test MSE Loss: {mse_loss.item():.6f}")
        print(f"Test RMSE Loss: {rmse_loss:.6f}")
        print(f"Test SMAPE Loss: {smape_loss:.6f}%")

        return mse_loss.item(), rmse_loss, smape_loss


## Simple MSE Loss function

In [19]:
params = {'n_hidden_layers': 5,
 'n_hidden_units': 219,
 'lr': 0.001400724556706755,
 'weight_decay': 1.4564281375374214e-08,
 'batch_size': 16}

# Train the model with the best hyperparameters
Phy_mlp = PhysicsMLP(
    N_INPUT_UNITS=train_dataset.__n_features_in__(),
    N_HIDDEN_LAYERS=params["n_hidden_layers"],
    N_HIDDEN_UNITS=params["n_hidden_units"],
    N_OUTPUT_UNITS=train_dataset.__n_features_out__(),
    loss_function="MSE",
)

# Create train & validation loaders with the best batch size
train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=params["batch_size"], shuffle=False)

# Train the model
Phy_mlp.train_model(train_loader, val_loader, epochs=50, lr=params["lr"], weight_decay=params["weight_decay"], device=device)



Epoch 1/50 - Train Loss: 0.063185 - Val Loss: 0.039295
Epoch 2/50 - Train Loss: 0.029938 - Val Loss: 0.057857
Epoch 3/50 - Train Loss: 0.027003 - Val Loss: 0.033442
Epoch 4/50 - Train Loss: 0.026025 - Val Loss: 0.037039
Epoch 5/50 - Train Loss: 0.021648 - Val Loss: 0.027958
Epoch 6/50 - Train Loss: 0.018225 - Val Loss: 0.025800
Epoch 7/50 - Train Loss: 0.016240 - Val Loss: 0.023938
Epoch 8/50 - Train Loss: 0.015933 - Val Loss: 0.022082
Epoch 9/50 - Train Loss: 0.015482 - Val Loss: 0.021290
Epoch 10/50 - Train Loss: 0.015093 - Val Loss: 0.021010
Epoch 11/50 - Train Loss: 0.015702 - Val Loss: 0.021218
Epoch 12/50 - Train Loss: 0.014756 - Val Loss: 0.022029
Epoch 13/50 - Train Loss: 0.014722 - Val Loss: 0.020772
Epoch 14/50 - Train Loss: 0.014679 - Val Loss: 0.021470
Epoch 15/50 - Train Loss: 0.014835 - Val Loss: 0.020759
Epoch 16/50 - Train Loss: 0.014754 - Val Loss: 0.022891
Epoch 17/50 - Train Loss: 0.015242 - Val Loss: 0.024430
Epoch 18/50 - Train Loss: 0.016656 - Val Loss: 0.021316
E

0.02075547818094492

In [24]:
import torch
import torch.nn as nn

def test_model(model, test_loader, device, min_value=None, max_value=None):
    """
    Evaluates the given model on a test dataset and computes MSE, RMSE, and SMAPE.

    Args:
        model (torch.nn.Module): The trained model.
        test_loader (DataLoader): DataLoader for the test dataset.
        device (torch.device): The device to run inference on.
        min_value (float, optional): Minimum value of the dataset (for denormalization).
        max_value (float, optional): Maximum value of the dataset (for denormalization).

    Returns:
        tuple: (mse_loss, rmse_loss, smape_loss) computed on the test set.
    """
    model.to(device)
    model.eval()
    mse_loss_fn = nn.MSELoss()
    rmse_loss = 0.0
    smape_loss = 0.0
    total_elements = 0
    epsilon = 1e-6  # To prevent division by zero

    with torch.no_grad():
        for u, y in test_loader:
            u, y = u.to(device), y.to(device)
            output = model(u)  # Forward pass

            # Compute SMAPE BEFORE denormalization
            abs_diff = torch.abs(y - output)
            sum_abs = torch.abs(y) + torch.abs(output) + epsilon  # Avoid division by zero
            smape_batch = torch.sum(2 * abs_diff / sum_abs).item()

            smape_loss += smape_batch
            total_elements += y.numel()  # Count total number of elements

            # Denormalize for RMSE and MSE calculation
            if min_value is not None and max_value is not None:
                output = output * (max_value - min_value) + min_value
                y = y * (max_value - min_value) + min_value

            mse_loss = mse_loss_fn(output, y)
            rmse_loss += torch.sqrt(mse_loss).item()

    # Final loss calculations
    rmse_loss /= len(test_loader)  # Average over batches
    smape_loss = (smape_loss / total_elements) * 100  # Average over all elements and convert to %

    print(f"Test MSE Loss: {mse_loss.item():.6f}")
    print(f"Test RMSE Loss: {rmse_loss:.6f}")
    print(f"Test SMAPE Loss: {smape_loss:.6f}%")

    return mse_loss.item(), rmse_loss, smape_loss

In [25]:
# best_model_baseline.load_state_dict(torch.load(f"{MODEL_PATH}/best_mlp_no2_baseline.pth"))
# best_model_baseline.eval()

# Create the DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)

# # Evaluate the model on the test dataset
df_minmax = pd.read_csv(MINMAX_PATH, sep=';')
min_value = df_minmax["min"].values
max_value = df_minmax["max"].values
test_model(Phy_mlp, test_loader, device, min_value, max_value)



Test MSE Loss: 148.015683
Test RMSE Loss: 14.104732
Test SMAPE Loss: 36.083454%


(148.01568267826852, 14.104732335989837, 36.083454114419446)